# Setup

Clone repo, install dependencies and check PyTorch and GPU.

In [1]:
!pip install --upgrade pip setuptools wheel
!git clone https://github.com/mikel-brostrom/yolo_tracking.git  # clone repo
%cd yolo_tracking
!pip install -e .

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 2.2.1+cu121 (CPU)


Tranform picture to video

In [ ]:
import cv2
import os
import natsort

image_folder = "data/frames"

images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]
images = natsort.natsorted(images)

print(f"Images size: {len(images)}")

height, width, _ = cv2.imread(os.path.join(image_folder, images[0])).shape

video = cv2.VideoWriter(
    "video.avi", cv2.VideoWriter_fourcc(*"xvid"), 30, (width, height)
)

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()

Images size: 1385


## Run inference on video

The ``cv2.imshow()`` and ``cv.imshow()`` functions from the [opencv-python](https://github.com/skvark/opencv-python) package are incompatible with Jupyter notebook; see https://github.com/jupyter/notebook/issues/3935.

Hence we chose to save it to file in this notebook. Locally you can use the ``--show-vid`` flag in order visualize the tracking in real-time

In [3]:
!python tracking/track.py --yolo-model yolov8l.pt --reid-model osnet_x0_25_msmt17.pt --tracking-method bytetrack --source video.avi --classes 41 --save --save-txt --conf 0.20

2024-04-15 01:25:26.534 | ERROR    | boxmot.utils.checks:check_packages:26 - The 'ultralytics@ git+https://github.com/mikel-brostrom/ultralytics.git' distribution was not found and is required by the application
2024-04-15 01:25:26.535 | WARNING  | boxmot.utils.checks:check_packages:29 - 
Missing packages: "ultralytics @ git+https://github.com/mikel-brostrom/ultralytics.git" 
Atempting installation...
2024-04-15 01:25:37.146 | SUCCESS  | boxmot.utils.checks:check_packages:35 - All the missing packages were installed successfully
100% 83.7M/83.7M [00:00<00:00, 139MB/s]

Traceback (most recent call last):
  File "/content/yolo_tracking/tracking/track.py", line 172, in <module>
    run(opt)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/content/yolo_tracking/tracking/track.py", line 101, in run
    for r in results:
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.p

Convert to single correct text file

In [ ]:
import glob
import os

result_path = glob.glob(os.path.join("/content/yolo_tracking/runs/track/exp/labels", "*.txt"))
dict_result = dict()

for path in result_path:
  index = int(path.replace("/content/yolo_tracking/runs/track/exp/labels/video_", "").replace(".txt", ""))
  current_result = ""

  with open(path) as f:
    lines = f.readlines()

    for line in lines:
      param = line.split(" ")
      if len(param) == 6:
        id = param[5].replace("\n", "")
        width = round(float(param[3]) * 1920)
        height = round(float(param[4]) * 1080)
        xmin = round((float(param[1]) * 1920) - (width / 2))
        ymin = round((float(param[2]) * 1080) - (height / 2))
        current_result += f"{index} {id} {xmin} {ymin} {width} {height}\n"

  dict_result[index] = current_result

with open("result.txt", 'w') as f:
  for key in range(1, len(images) + 1):
    if key in dict_result:
      f.writelines(dict_result[key])

# Show results

https://stackoverflow.com/questions/60977179/how-to-play-avi-file-in-google-colab

https://stackoverflow.com/questions/57377185/how-play-mp4-video-in-google-colab

Compress the video file to a format supported by Google Colab (mpeg4 (native) -> h264 (libx264))

In [ ]:
!ffmpeg -i /content/yolo_tracking/runs/track/exp/video.avi -vf fps=30 -vcodec libx264 output.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

Get the file content into data_url

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('output.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

Display it with HTML

In [ ]:
HTML("""
<video width="640" controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

# Evaluate

In [2]:
!python tracking/val.py --tracking-method bytetrack --benchmark MOT17 --yolo-model yolov8l.pt --reid-model osnet_x0_25_msmt17.pt --imgsz 320 --conf 0.2

Traceback (most recent call last):
  File "/content/yolo_tracking/tracking/val.py", line 7, in <module>
    from tracking.utils import (
  File "/content/yolo_tracking/tracking/utils.py", line 5, in <module>
    from ultralytics.utils import ops
ModuleNotFoundError: No module named 'ultralytics'
